In [25]:
import sys
print(sys.executable)


e:\anaconda3\envs\pens_clean\python.exe


In [26]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import pickle
from tqdm import tqdm
import re
import json
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from collections import Counter

In [27]:
from model import * # 这里\UserEncoder\model.py里要用from modules
from data import *
from utils import *

In [28]:
import os
os. environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda:0')

## Train

- **set params**

In [29]:
lr = 0.0001
batch_size=128

- **load train data**

In [30]:
news_vert = np.load('../../data2/news_vert.npy')
news_title = np.load('../../data2/news_title.npy')
news_body = np.load('../../data2/news_body.npy')

In [ ]:
with open('../../data2/TrainUsers.pkl', 'rb') as f:
    TrainUsers = pickle.load(f)
with open('../../data2/TrainSamples.pkl', 'rb') as f:
    TrainSamples = pickle.load(f)

- **load model**

In [ ]:
with open('../../data2/dict.pkl', 'rb') as f:
    _,category_dict,word_dict = pickle.load(f)
with open('../../data2/news.pkl', 'rb') as f:
    news = pickle.load(f)
embedding_matrix = np.load('../../data2/embedding_matrix.npy')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = NRMS(embedding_matrix)
model = model.to(device)

- **begin training**

In [ ]:
def acc(y_true, y_hat):
    y_hat = torch.argmax(y_hat, dim=-1)
    tot = y_true.shape[0]
    hit = torch.sum(y_true == y_hat)
    return hit.data.float() * 1.0 / tot

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0001) #lr = 0.0001

In [24]:
min_train_loss = 100.0
for ep in range(1,4):
    loss = 0.0
    accuary = 0.0
    cnt = 1
    dset = TrainDataset(TrainUsers, TrainSamples, news_title, news_vert, news_body)
    # ! 设置 `num_workers` 参数来使用多进程/多线程数据加载，提高数据加载速度，减少 GPU 等待数据的时间。
    # ! `pin_memory=True`：将数据加载到固定内存（pinned memory）中，固定内存是 GPU 和 CPU 之间数据传输的高效内存区域。加速数据从 CPU 内存传输到 GPU 显存的过程
    data_loader = DataLoader(dset, batch_size=128, collate_fn=collate_fn, shuffle=True, num_workers=20, pin_memory=True)
    tqdm_util = tqdm(data_loader)
    for user_feature, news_feature, label in tqdm_util: 
        user_feature = [i.to(device) for i in user_feature]
        news_feature = [i.to(device) for i in news_feature]
        label = label.to(device)
        bz_loss, y_hat = model(user_feature, news_feature, label)
        loss += bz_loss.data.float()
        accuary += acc(label, y_hat)

        optimizer.zero_grad()
        bz_loss.backward()
        optimizer.step()

        if cnt % 10 == 0:
            tqdm_util.set_description('ed: {}, train_loss: {:.5f}, acc: {:.5f}'.format(cnt * batch_size, loss.data / cnt, accuary / cnt))
        cnt += 1
    loss /= cnt
    print(ep, loss)
    torch.save(model.state_dict(), '../../runs/userencoder/NAML-{}.pkl'.format(ep))


  0%|          | 0/5963 [02:00<?, ?it/s]


MemoryError: 

## Test

- **load test data**

In [ ]:
with open('../../data2/ValidUsers.pkl', 'rb') as f:
    ValidUsers = pickle.load(f)
with open('../../data2/ValidSamples.pkl', 'rb') as f:
    ValidSamples = pickle.load(f)

In [ ]:
for ep in range(1,4):
    model = NRMS(embedding_matrix)
    model = model.to(device)
    model.load_state_dict(torch.load('../../runs/userencoder/NAML-{}.pkl'.format(ep)))
    model.eval() 
    # save new embedding matrix
    np.save('../../data2/embedding_matrix{}.npy'.format(ep), model.embed.weight.data.cpu().numpy())
    

    n_dset = news_dataset(news_title, news_vert, news_body)
    news_data_loader = DataLoader(n_dset, batch_size=512, collate_fn=news_collate_fn, shuffle=False, num_workers=20, pin_memory=True)

      
    news_scoring = []
    torch.cuda.empty_cache()
    with torch.no_grad():
        for news_feature in tqdm(news_data_loader): 
            news_feature = [i.to(device) for i in news_feature]
            news_vec = model.news_encoder(news_feature)
            news_vec = news_vec.to(torch.device("cpu")).detach().numpy()
            news_scoring.extend(news_vec)
    news_scoring = np.array(news_scoring)
    np.save('../../data2/news_scoring{}.npy'.format(ep), news_scoring)
    
    u_dset = UserDataset(news_scoring, ValidUsers)
    user_data_loader = DataLoader(u_dset, batch_size=128, shuffle=False, num_workers=20, pin_memory=True)

    user_scoring = []
    with torch.no_grad():
        for user_feature in tqdm(user_data_loader): 
            user_feature = user_feature.to(device)
            user_vec = model.user_encoder(user_feature)
            user_vec = user_vec.to(torch.device("cpu")).detach().numpy()
            user_scoring.extend(user_vec)
    user_scoring = np.array(user_scoring)
    np.save('../../data2/global_user_embed{}.npy'.format(ep),np.mean(user_scoring,axis=0))
    g = evaluate(user_scoring,news_scoring, ValidSamples)
    print(ep)
    print('AUC\t', 'MRR\t', 'nDCG5\t', 'nDCG10\t','CTR1\t','CTR10\t')
    print(g)

100%|██████████| 100000/100000 [02:25<00:00, 688.61it/s]


0 0.0
1
AUC	 MRR	 nDCG5	 nDCG10	 CTR1	 CTR10	
(0.637413011688109, 0.22836874347959216, 0.2489757536701546, 0.32785646035237614, 0.14377, 0.11723933333333335)


100%|██████████| 100000/100000 [02:25<00:00, 686.24it/s]


0 0.0
2
AUC	 MRR	 nDCG5	 nDCG10	 CTR1	 CTR10	
(0.6429802283551654, 0.23512888237734772, 0.2589661235268397, 0.33673295878516, 0.15212, 0.12148333333333332)


100%|██████████| 100000/100000 [02:26<00:00, 682.57it/s]

0 0.0
3
AUC	 MRR	 nDCG5	 nDCG10	 CTR1	 CTR10	
(0.6453690602233674, 0.2365845148820153, 0.26123821374720474, 0.33878635525601314, 0.15295, 0.12263333333333332)
